In [ ]:
path_to_muograph = "../"
import sys
sys.path.insert(1, path_to_muograph)

![alternative text](../images/muograph_logo.png)


**<h1><center>Binned Clustered Algorithm: tutorial</center></h1>**

The `BCA` class is used to compute voxelized **scattering density predictions** based on the **Binned Clustered Algorithm** (A binned clustering algorithm  to detect high-Z material using cosmic muons, 2013 JINST 8 P10013, (http://iopscience.iop.org/1748-0221/8/10/P10013))

### **I - Initialization** 

First, let's instanciante the `TrackingMST` class. At first, we will consider a detector with perfect **spatial resolution** and **efficiency**:

In [ ]:
csv_file = '../muograph/data/iron_barrel/barrel_and_cubes_scattering.csv'

from muograph.hits.hits import Hits

hits_in = Hits(
    csv_filename = csv_file,  # The csv file
    plane_labels = (0, 1, 2),  # Include hits from first planes 0, 1 and 2
    input_unit = "mm",  # The distance unit in the csv file.
)
hits_out = Hits(
    csv_filename = csv_file,  # The csv file
    plane_labels = (3, 4, 5),  # Include hits from first planes 0, 1 and 2
    input_unit = "mm",  # The distance unit in the csv file.
)

hits_in, hits_out

In [ ]:
from muograph.tracking.tracking import Tracking

tracks_in = Tracking(label="above", hits=hits_in)
tracks_out = Tracking(label="below", hits=hits_out)

tracks_in, tracks_out


In [ ]:
output_dir = "../output_test/"
from muograph.tracking.tracking import TrackingMST
mst = TrackingMST(trackings=(tracks_in, tracks_out))
mst

The `BCA` class takes the following **arguments**:


- `tracking (Optional[TrackingMST])`: Instance of the TrackingMST class.

- `voi (Volume)`: Instance of the Volume class.

- `output_dir (Optional[str])`: Path to a directory where to save POCA attributes in a hdf5 file.

In [ ]:
from muograph.volume.volume import Volume
voi = Volume(
    position = (0, 0, -1200), # The position of the Volume of interest.
    dimension = (1000, 600, 600), # The dimensions of the Volume of interest.
    voxel_width = 20) # The voxel size.

In [ ]:
from muograph.reconstruction.binned_clustered import BCA
bca = BCA(tracking=mst, voi=voi, output_dir="../output_test/")

### I - **BCA scattering density predictions**

The **BCA algorithm** is designed to compute voxel-wize **scattering density predictions**, using **POCA**, **scattering angle** and muon **momentum** information (if available).

**Initialization:**

 - Associate an **empty list** $L$ to **each voxel**.

 - **Filters out** muons with **scattering angle** $\delta\theta$ outside of the desired range `dtheta_range`.

 - **Filters out** muons with **momentum** $p$ outside of the desired range `momentum_range` (if momentum is available).

**Voxel scan:**

- For each voxel $v$:
  - Only keep the POCA points having the $N_{max}$ largest scattering angles (`n_max_per_vox`).
  - If the number of POCA points in $v$ is greater than $N_{min}$ (`n_min_per_vox`):
    - For each **pair** of **POCA** points $i, j$ in $v$:
      - Computes $m_{ij} = \frac{d_{ij}}{(\Delta\theta_i p_i) * (\Delta\theta_j p_j)}$, where $d_{ij}$ is the geometrical distance between the POCA $_i$ and POCA $_j$, $\Delta\theta$ the scattering angle and $p$ the momentum.
      - Append $f(m_{ij})$ to the voxel's list $L_v$, where $f$ is the `metric_method` function (natural $log$ by default).
  - Else append 0 to $L_v$.

**At the end of the scan**

For each voxel $v$:

- Computes the **final** voxel's **score** $S_v$ as a function of the voxel's list $L_v$: $S_v = g(L_v)$, where $g$ is the `score_method`. By default, $g$  the **median function**. 

In [ ]:
bca.bca_params

Similarly to `POCA.n_poca_per_vox`, the BCA scattering density predictions `BCA.xyz_voxel_pred` is a  $(n_x, n_y, n_z)$ array of `floats`, where $n_i$ is the number of voxels along the $i$ direction.

In [ ]:
bca.xyz_voxel_pred, bca.xyz_voxel_pred.shape

Let's have a look at the BCA scattering density predictions. They are stored in the `BCA.xyz_voxel_pred` feature.

In [ ]:
from muograph.plotting.plotting import plot_hist
plot_hist(bca.xyz_voxel_pred.ravel(), xlabel="BCA score [a.u]", logy=True)

As the `BCA` class also **inherits** from the `VoxelPlotting` class, it comes with dedicated methods to **plot voxelized data**.

In [ ]:
# Choose a dimension to project along (0 = x, 1 = y, 2 = z)
dim = 2 

bca.plot_pred_slice(
    voi=voi,  # The volume of interest
    xyz_voxel_preds=bca.xyz_voxel_pred,  # The voxelized predictions to plot
    voi_slice=(0, voi.n_vox_xyz[dim]-1), # Choose a range of voxels to plot along the desired direction
    dim = dim, # The dimension to project along
    figname = "../output_test/bca_default_preds_all_volume", # The name of the figure to save
    pred_label="BCA score",
    pred_unit="\n[a.u]",
)

In [ ]:
# Choose a dimension to project along (0 = x, 1 = y, 2 = z)
dim = 1

bca.plot_pred_slice(
    voi=voi,  # The volume of interest
    xyz_voxel_preds=bca.xyz_voxel_pred,  # The voxelized predictions to plot
    voi_slice=(0, voi.n_vox_xyz[dim]-1), # Choose a range of voxels to plot along the desired direction
    dim = dim, # The dimension to project along
    figname = "../output_test/bca_default_preds_all_volume", # The name of the figure to save
    pred_label="BCA score",
    pred_unit="\n[a.u]",
)

In [ ]:
# Choose a dimension to project along (0 = x, 1 = y, 2 = z)
dim = 0

bca.plot_pred_slice(
    voi=voi,  # The volume of interest
    xyz_voxel_preds=bca.xyz_voxel_pred,  # The voxelized predictions to plot
    voi_slice=(0, voi.n_vox_xyz[dim]-1), # Choose a range of voxels to plot along the desired direction
    dim = dim, # The dimension to project along
    figname = "../output_test/bca_default_preds_all_volume", # The name of the figure to save
    pred_label="BCA score",
    pred_unit="\n[a.u]",
)

In [ ]:
# Choose a dimension to project along (0 = x, 1 = y, 2 = z)
dim = 2

bca.plot_pred_by_slice(
    voi=voi, # The volume of interest
    xyz_voxel_preds=bca.xyz_voxel_pred,  # The voxelized predictions to plot
    voi_slice=(0, voi.n_vox_xyz[dim]-1), # Choose a range of voxels to plot along the desired direction
    dim=dim,  # The dimension to project along
    nslice_per_plot=2, # The number of slice per plot
    ncols=4, # The number of columns of the plot
    pred_unit="[a.u]",
    pred_label="BCA score",
    figname = "../output_test/bca_default_preds_slice"
)


In [ ]:
# Choose a dimension to project along (0 = x, 1 = y, 2 = z)
dim = 1

bca.plot_pred_by_slice(
    voi=voi, # The volume of interest
    xyz_voxel_preds=bca.xyz_voxel_pred,  # The voxelized predictions to plot
    voi_slice=(0, voi.n_vox_xyz[dim]-1), # Choose a range of voxels to plot along the desired direction
    dim=dim,  # The dimension to project along
    nslice_per_plot=2, # The number of slice per plot
    ncols=4, # The number of columns of the plot
    pred_unit="[a.u]",
    pred_label="BCA score",
    figname = "../output_test/bca_default_preds_slice"
)

In [ ]:
# Choose a dimension to project along (0 = x, 1 = y, 2 = z)
dim = 0

bca.plot_pred_by_slice(
    voi=voi, # The volume of interest
    xyz_voxel_preds=bca.xyz_voxel_pred,  # The voxelized predictions to plot
    voi_slice=(0, voi.n_vox_xyz[dim]-1), # Choose a range of voxels to plot along the desired direction
    dim=dim,  # The dimension to project along
    nslice_per_plot=2, # The number of slice per plot
    ncols=4, # The number of columns of the plot
    pred_unit="[a.u]",
    pred_label="BCA score",
    figname = "../output_test/bca_default_preds_slice"
)


The **parameters** of the **algorithm** are stored in the `bca_params` attribute:

In [ ]:
bca.bca_params

If the parameters of the algo. are modified, the `xyz_voxel_pred` are automatically re-computed.

Let's set the `use_p` parameter to `True`, and **use** the **momentum information** in the **computation** of the scattering density **predictions**.

In [ ]:
from functools import partial
import torch
import math

bca.bca_params = {
    "n_max_per_vox": 50,
    "n_min_per_vox": 2,
    "score_method": partial(torch.quantile, q=0.7),
    "metric_method": partial(torch.log), 
    "p_range": (0.0, 10_000_000),  # MeV
    "dtheta_range": (0.1 * math.pi / 180, math.pi / 3),
    "use_p": True,
    }

Let's have a look at the new predictions:

In [ ]:
from muograph.plotting.plotting import plot_hist
plot_hist(bca.xyz_voxel_pred.ravel(), xlabel="BCA score [a.u]", logy=True)

In [ ]:
dim = 2

bca.plot_pred_by_slice(
    voi=voi, # The volume of interest
    xyz_voxel_preds=bca.xyz_voxel_pred,  # The voxelized predictions to plot
    voi_slice=(0, voi.n_vox_xyz[dim]-1), # Choose a range of voxels to plot along the desired direction
    dim=dim,  # The dimension to project along
    nslice_per_plot=2, # The number of slice per plot
    ncols=4, # The number of columns of the plot
    pred_unit="[a.u]",
    pred_label="BCA score",
    reverse=True, # Reverse color scale
    figname = "../output_test/bca_default_preds_use_p_slice"
)

In [ ]:
# Choose a dimension to project along (0 = x, 1 = y, 2 = z)
dim = 2 

bca.plot_pred_slice(
    voi=voi,  # The volume of interest
    xyz_voxel_preds=bca.xyz_voxel_pred,  # The voxelized predictions to plot
    voi_slice=(0, voi.n_vox_xyz[dim]-1), # Choose a range of voxels to plot along the desired direction
    dim = dim, # The dimension to project along
    figname = "../output_test/bca_default_preds_use_p_all_volume", # The name of the figure to save
    pred_label="BCA score",
    pred_unit="\n[a.u]",
    reverse=True
)